In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [10]:
train_dir = 'C:/Users/Alan Mathias/projeto_mosquito_V2/spectrograms_3ch_dataset/train'
test_dir = 'C:/Users/Alan Mathias/projeto_mosquito_V2/spectrograms_3ch_dataset/test'
val_dir = 'C:/Users/Alan Mathias/projeto_mosquito_V2/spectrograms_3ch_dataset/val'

img_width, img_height = 64, 64
num_classes = 2



In [12]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_width, img_height), batch_size=32, class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(img_width, img_height), batch_size=32, class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(img_width, img_height), batch_size=32, class_mode='categorical')


Found 5187 images belonging to 2 classes.
Found 1481 images belonging to 2 classes.
Found 743 images belonging to 2 classes.


In [13]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model_comum = Sequential()
model_comum.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 3)))
model_comum.add(MaxPooling2D((2, 2)))

model_comum.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model_comum.add(MaxPooling2D((2, 2)))

model_comum.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model_comum.add(MaxPooling2D((2, 2)))

model_comum.add(Flatten())
model_comum.add(Dense(128, activation='relu'))
model_comum.add(Dense(num_classes, activation='sigmoid'))
model_comum.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 8, 8, 64)        

In [14]:
model_comum.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model_comum.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
163/163 [==============================] - 38s 224ms/step - loss: 0.0740 - accuracy: 0.9803 - val_loss: 0.0015 - val_accuracy: 1.0000
Epoch 2/10
163/163 [==============================] - 36s 221ms/step - loss: 0.0096 - accuracy: 0.9969 - val_loss: 0.0012 - val_accuracy: 1.0000
Epoch 3/10
163/163 [==============================] - 36s 222ms/step - loss: 0.0156 - accuracy: 0.9960 - val_loss: 6.0537e-04 - val_accuracy: 1.0000
Epoch 4/10
163/163 [==============================] - 37s 225ms/step - loss: 7.6631e-04 - accuracy: 0.9998 - val_loss: 1.1009e-04 - val_accuracy: 1.0000
Epoch 5/10
163/163 [==============================] - 37s 227ms/step - loss: 6.9195e-04 - accuracy: 0.9996 - val_loss: 2.9776e-06 - val_accuracy: 1.0000
Epoch 6/10
163/163 [==============================] - 39s 239ms/step - loss: 3.9226e-04 - accuracy: 0.9998 - val_loss: 2.5321e-04 - val_accuracy: 1.0000
Epoch 7/10
163/163 [==============================] - 38s 231ms/step - loss: 6.4813e-05 - accuracy: 1.

In [15]:
# Compile o modelo com as mesmas configurações usadas no treinamento
model_comum.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Avalie o modelo no conjunto de teste
test_loss, test_acc = model_comum.evaluate(test_generator)

print('Perda no conjunto de teste:', test_loss)
print('Acurácia no conjunto de teste:', test_acc)


47/47 [==============================] - 16s 334ms/step - loss: 0.0013 - accuracy: 0.9993
Perda no conjunto de teste: 0.0012651412980630994
Acurácia no conjunto de teste: 0.9993247985839844


In [17]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Gerando as previsões para os dados de teste
y_pred = np.argmax(model_comum.predict(test_generator), axis=-1)

# Obtendo os rótulos verdadeiros dos dados de teste
y_true = test_generator.classes

# Criando a matriz de confusão
cm = confusion_matrix(y_true, y_pred)

print(cm)


47/47 [==============================] - 4s 89ms/step
[[392 355]
 [354 380]]


In [4]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))


for layer in vgg_model.layers:
    layer.trainable = False

model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='sigmoid'))


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 514       
                                                                 
Total params: 16,812,610
Trainable params: 2,097,922
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
  3/163 [..............................] - ETA: 20:55 - loss: 0.9382 - accuracy: 0.4688